# Installs

In [ ]:
!pip install gradio

# Imports

In [19]:
from google.colab import drive
from pathlib import Path
from fastai.vision.all import *
from fastai.vision.core import PILBase
import PIL
from PIL import Image
from PIL import ImageOps
import gradio as gr

# Define Enhancer

In [ ]:
def enhance_image(img: PIL.Image.Image):
    # Convert to grayscale
    img = img.convert('L')
    
    # Apply histogram equalization to improve contrast
    enhanced_img = ImageOps.equalize(img)

    return enhanced_img

In [ ]:
class EnhanceImage(Transform):
    def __init__(self):
        pass

    def encodes(self, x: PIL.Image.Image):
        return enhance_image(x)

    def decodes(self, x: PIL.Image.Image):
        return x

# Load Learner

In [15]:
drive.mount('/content/gdrive')
DATA = Path('/content/gdrive/MyDrive/DAT255/x-ray')
DATA.mkdir(exist_ok=True, parents=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [22]:
learn = load_learner(DATA/'model_resnet50_v2.pkl')

# Create a gradio app

In [23]:
labels = ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia', 'No Finding']

In [28]:
def predict(image: Image.Image):
    img_fastai = PILImage.create(image)

    # Apply the EnhanceImage transform
    enhancer = EnhanceImage()
    img_fastai = enhancer(img_fastai)

    img_fastai = img_fastai.resize((460, 460)).convert("RGB")

    _, _, logits = learn.predict(img_fastai)
    logits = logits.numpy()

    # Apply sigmoid function to logits
    probs = 1 / (1 + np.exp(-logits))

    return {labels[i]: float(probs[i]) for i in range(len(probs))}

In [29]:
iface = gr.Interface(
    fn=predict,
    inputs=gr.inputs.Image(shape=(460, 460), label="Upload a Chest X-Ray Image"),
    outputs=gr.outputs.Label(num_top_classes=5, label="Predicted Conditions"),  # Adjust the value to the number of top classes you'd like to display
    title="Chest X-Ray Disease Prediction",
)

/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:257: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


In [30]:
iface.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>